# Quantitative Research Workflows
This notebook contains organized workflows for running various quantitative operations including Jenkins workflows, HECM reports, CRT operations, and tracking reports.

## Configuration
Update the global variables below to customize runs across all sections.

In [3]:
# Global Configuration Variables
import os
from datetime import datetime, timedelta

# Primary date configuration - Update as needed (default: most recent business day)
# Set AS_OF_DATE to the most recent business day (excluding weekends)
today = datetime.today() - timedelta(days=1)
while today.weekday() > 4:  # Saturday: 5, Sunday: 6
    today -= timedelta(days=1)
AS_OF_DATE = today.strftime("%Y%m%d")  # Format: YYYYMMDD

# Email configuration
EMAIL_RECIPIENTS = "hzeng@libremax.com"
EMAIL_RECIPIENTS_MULTI = "hzeng@libremax.com,Ksam@libremax.com"

# Common paths
GIT_LMQR_PATH = "C:\\Git\\LMQR"
GIT_LMQR_PROD_PATH = "C:\\Git\\LMQR_prod"
GIT_LMSIM_PATH = "C:\\Git\\LMSim"
GIT_LMSIMDATA_PROD_PATH = "C:\\Git\\LMSimData_prod"

# Ray cluster configuration
RAY_CLUSTER = "east-spot"
NUM_WORKERS = 16
NUM_THREADS = 7

# Common scenario sets
BASE_SCENARIOS = "Base"
TRACKING_SCENARIOS = "Base,SuperBear,SuperBull,ParallelDn300,ParallelUp300"
FULL_SCENARIOS = "Base,SuperBear,SuperBull,ParallelDn300,ParallelUp300"

# Date range calculations - Revised for improved flexibility, doc clarity, and future extensibility
def get_date_ranges(as_of_date, unload_start="20240101", unload_end_month="09"):
    """
    Generate common date ranges based on as_of_date.
    
    Parameters:
        as_of_date (str): Reference date in 'YYYYMMDD' format.
        unload_start (str): Starting date for unload, default '20240101'.
        unload_end_month (str): Month (MM) to use for 'unload_end', default '09' (September).
        
    Returns:
        dict: Dictionary containing 'start_of_year', 'unload_start', and 'unload_end'.
    """
    yearmonth = int(as_of_date[:6])
    # Compose unload_end in format YYYYMM01, where MM is unload_end_month
    unload_end = f"{yearmonth}01"
    return {
        "unload_start": unload_start,
        "unload_end": unload_end
    }

date_ranges = get_date_ranges(AS_OF_DATE)
print(f">>> Configuration loaded for date: {AS_OF_DATE}")
print(f">>> Date ranges: {date_ranges}")

>>> Configuration loaded for date: 20260203
>>> Date ranges: {'unload_start': '20240101', 'unload_end': '20260201'}


## 1. Jenkins Workflows
Monthly Residential Tracking Workflow components

In [48]:
# Jenkins Workflow Steps (quant-Monthly-ResiTracking-Workflow)
jenkins_workflows = [
    "1. quant-CRT-PseudoPools - Generate pseudo pools",
    "2. quant-DailySimDataUpdate - Write the flat files", 
    "3. quant-Resi-Transition-History - Calculate the historical transition matrix",
    "4. quant-Resi-Transition-Projection - Running Projection",
    "5. quant-Resi-Transition-Tracking - Running historical tracking",
    "6. quant-Monthly-Tracking-Report - Generate excel report from tracking",
    "7. quant-tracking-report-recache-workflow - Update Galileo"
]

for workflow in jenkins_workflows:
    print(workflow)

1. quant-CRT-PseudoPools - Generate pseudo pools
2. quant-DailySimDataUpdate - Write the flat files
3. quant-Resi-Transition-History - Calculate the historical transition matrix
4. quant-Resi-Transition-Projection - Running Projection
5. quant-Resi-Transition-Tracking - Running historical tracking
6. quant-Monthly-Tracking-Report - Generate excel report from tracking
7. quant-tracking-report-recache-workflow - Update Galileo


## 2. HECM Report Automation
Monthly HECM report generation with email notifications

In [49]:
# HECM Monthly Report - Single Month
def generate_hecm_command(collatdate, skip_data_refresh=False, recipients=EMAIL_RECIPIENTS):
    cmd = f"python hecm_monthly_automation.py --collatdate {collatdate} --send-email --email-recipients \"{recipients}\""
    if skip_data_refresh:
        cmd += " --skip-data-refresh"
    return cmd

# Generate HECM report for current date
hecm_current = generate_hecm_command(AS_OF_DATE[:6], skip_data_refresh=True)
print("HECM Report Command:")
print(hecm_current)

HECM Report Command:
python hecm_monthly_automation.py --collatdate 202601 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh


In [50]:
# HECM Batch Processing - Multiple Months
hecm_months = ["202411", "202412", "202501", "202502", "202503", "202504", "202505", "202506"]

print("HECM Batch Commands:")
for month in hecm_months:
    cmd = generate_hecm_command(month, skip_data_refresh=True)
    print(cmd)
    print()

HECM Batch Commands:
python hecm_monthly_automation.py --collatdate 202411 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202412 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202501 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202502 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202503 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202504 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202505 --send-email --email-recipients "hzeng@libremax.com" --skip-data-refresh

python hecm_monthly_automation.py --collatdate 202506 --send-email --email-recipien

---
# 📁 FLAT FILE GENERATION
Update data sources and generate flat files for all deal types

---

## 3. CRT Pseudo Deal Creation
Creating and managing CRT pseudo deals for STACR and CAS

In [51]:
# CRT Pseudo Deal Commands
crt_pseudo_commands = [
    f"python agencydata\\wh_crt_pseudo_sample.py",
    f"python agencydata\\wh_crt_pseudo_sample.py --deal_type STACR_PSEUDO --rebuild",
    f"python agencydata\\wh_crt_pseudo_sample.py --deal_type CAS_PSEUDO --rebuild"
]

print("CRT Pseudo Deal Creation:")
for cmd in crt_pseudo_commands:
    print(cmd)
    print()

CRT Pseudo Deal Creation:
python agencydata\wh_crt_pseudo_sample.py

python agencydata\wh_crt_pseudo_sample.py --deal_type STACR_PSEUDO --rebuild

python agencydata\wh_crt_pseudo_sample.py --deal_type CAS_PSEUDO --rebuild



## 4. CRT and LP Data Updates
Commands for updating flat files from various data sources

In [52]:
def generate_wh_crt_update(
    deal_type: str,
    unload_start: str = None,
    unload_end: str = None,
    deal_list: str = None,
    unload_all_collat: bool = False,
    unload_first_collat: bool = False,
    skip_update: bool = False,
    force_update: bool = False,
    skip_insert: bool = False,
    force_insert: bool = False,
    skip_unload: bool = False,
    force_download: bool = False,
    skip_save_stats: bool = False,
    force_save_stats: bool = False,
    skip_save_trans: bool = False,
    force_save_trans: bool = False,
    stats_type: str = "pool",
    additional_flags: str = "",
) -> str:
    # --- Validation ---
    valid_stats = {"pool", "pool_status", "adv_pool"}
    if stats_type not in valid_stats:
        raise ValueError(f"Invalid stats_type: {stats_type}, must be one of {valid_stats}")
    
    def bool_flag(flag_name: str, flag_value: bool) -> str:
        return f" --{flag_name}" if flag_value else ""

    # --- Base Command ---
    base_cmd = f"python agencydata\\wh_crt_update.py --log --stats_type {stats_type}"

    # --- Deal list vs date range ---
    deal_range_args = ""
    if deal_type:
        deal_range_args += f" --deal_type {deal_type}"
    if deal_list:
        deal_range_args += f" --deal_list \"{deal_list}\""
    if not unload_all_collat:
        if unload_start:
            deal_range_args += f" --unload_start_date {unload_start}"
        if unload_end:
            deal_range_args += f" --unload_end_date {unload_end}"

    # --- Flag Set ---
    flags = (
        bool_flag("unload_all_collat", unload_all_collat)
        + bool_flag("unload_first_collat", unload_first_collat)
        + bool_flag("skip_update", skip_update)
        + bool_flag("force_update", force_update)
        + bool_flag("skip_insert", skip_insert)
        + bool_flag("force_insert", force_insert)
        + bool_flag("skip_unload", skip_unload)
        + bool_flag("force_download", force_download)
        + bool_flag("skip_save_stats", skip_save_stats)
        + bool_flag("force_save_stats", force_save_stats)
        + bool_flag("skip_save_trans", skip_save_trans)
        + bool_flag("force_save_trans", force_save_trans)
    )

    # --- Final Command ---
    cmd = f"{base_cmd}{deal_range_args}{flags} {additional_flags}".strip()
    return " ".join(cmd.split())  # clean up any double spaces


# ============================================================
# Example usage
# ============================================================
deal_type = "CAS"  # or STACR_PSEUDO, CAS_PSEUDO CAS STACR MIR FNM
deal_list = "STACR 2019-DNA3"  # "STACR 2019-DNA3"
unload_start = "20250601"
unload_end = "20251001"
stats_type = "pool"  # pool, pool_status, adv_pool
save_trans = False
unload_all_collat = False
force_update = False

cmd = generate_wh_crt_update(
    deal_type=deal_type,
    deal_list=deal_list,            # prioritize deal_list over date range
    unload_all_collat=unload_all_collat,
    skip_update=False,
    skip_insert=False,
    unload_start=unload_start,
    unload_end=unload_end,
    force_save_stats=True,
    force_insert=False,
    stats_type=stats_type,
    skip_unload=True,
    force_download=False,
    force_save_trans=save_trans,
    force_update=force_update
)

print(cmd)

python agencydata\wh_crt_update.py --log --stats_type pool --deal_type CAS --deal_list "STACR 2019-DNA3" --unload_start_date 20250601 --unload_end_date 20251001 --skip_unload --force_save_stats


In [53]:
# Ad-hoc CRT/LP Update Examples
print("LP Update for NONQM:")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"NONQM\" --deal_list \"VERUS 2022-1\" --force_save_stats --force_skip_trans")
print()

print("LP Update with Deal Download Date Range:")
print("Python agencydata/wh_lp_update.py --log --skip_unload --deal_type \"NONQM\" --deal_list \"VERUS 2022-1\" --deal_downloaddate_start 20250101 --deal_downloaddate_end 20251001 --skip_unload --force_skip_stats --force_skip_trans")
print()

print("CRT Update for Specific MIR Deal:")
print("python agencydata\\wh_crt_update.py --stats_type pool --deal_list \"HMIR 2023-1\" --skip_unload --skip_insert --force_save_stats")
print()

print("CRT MIR Update with Force Download:")
print("python agencydata\\wh_crt_update.py --log --force_download --deal_type MIR --deal_list \"BMIR 2025-1\" --force_save_stats --unload_all_collateral --skip_update --force_insert")
print()

LP Update for NONQM:
Python agencydata/wh_lp_update.py --skip_unload --deal_type "NONQM" --deal_list "VERUS 2022-1" --force_save_stats --force_skip_trans

LP Update with Deal Download Date Range:
Python agencydata/wh_lp_update.py --log --skip_unload --deal_type "NONQM" --deal_list "VERUS 2022-1" --deal_downloaddate_start 20250101 --deal_downloaddate_end 20251001 --skip_unload --force_skip_stats --force_skip_trans

CRT Update for Specific MIR Deal:
python agencydata\wh_crt_update.py --stats_type pool --deal_list "HMIR 2023-1" --skip_unload --skip_insert --force_save_stats

CRT MIR Update with Force Download:
python agencydata\wh_crt_update.py --log --force_download --deal_type MIR --deal_list "BMIR 2025-1" --force_save_stats --unload_all_collateral --skip_update --force_insert



In [ ]:
# CRT and LP Update Commands (all collateral, force download, skip update/insert, force save stats)
crt_update_types = ["CAS", "STACR", "MIR", "CAS_PSEUDO", "STACR_PSEUDO"]
crt_update_cmds = [
    f"python agencydata\\wh_crt_update.py --log --force_download --deal_type {dt} --unload_all_collat --skip_update --skip_insert --force_save_stats"
    for dt in crt_update_types
]

lp_update_cmd = "python agencydata\\wh_lp_update.py --force_unload --deal_type JUMBO2_0 --unload_all_collat"

deal_list = ['JUMBO AGE >=60', 'JUMBO AGE [0-7)', 'JUMBO AGE [13-25)', 'JUMBO AGE [25-37)', 'JUMBO AGE [37-60)', 'JUMBO AGE [7-13)', 'JUMBO FICO >=800', 'JUMBO FICO [0-725)', 'JUMBO FICO [725-750)', 'JUMBO FICO [750-775)', 'JUMBO FICO [775-800)', 'JUMBO UPDLTV >=80', 'JUMBO UPDLTV [0-50)', 'JUMBO UPDLTV [50-60)', 'JUMBO UPDLTV [60-65)', 'JUMBO UPDLTV [65-70)', 'JUMBO UPDLTV [70-75)', 'JUMBO UPDLTV [75-80)', 'JUMBO WAC >=7.75', 'JUMBO WAC [0-5)', 'JUMBO WAC [5-5.5)', 'JUMBO WAC [5.5-6)', 'JUMBO WAC [6-6.25)', 'JUMBO WAC [6.25-6.5)', 'JUMBO WAC [6.5-6.75)', 'JUMBO WAC [6.75-7)', 'JUMBO WAC [7-7.25)', 'JUMBO WAC [7.25-7.5)', 'JUMBO WAC [7.5-7.75)']
deal_downloaddate_start = "20250901"
deal_downloaddate_end = "20251001"
deal_type = "JUMBO2_0" # JUMBO2_0_PSEUDO
deal_list = "CHASE 2024-6"

if deal_list is not None:
    deal_list_part = f'--deal_list "{deal_list}"'
else:
    deal_list_part = ""

# Now assemble the full command with *one* f-string per segment
lp_update_cmd = (
    "python agencydata\\wh_lp_update.py "
    "--log --force_unload "
    f"--deal_type {deal_type} "
    f"{deal_list_part} "
    f"--deal_downloaddate_start {deal_downloaddate_start} "
    f"--deal_downloaddate_end {deal_downloaddate_end} "
    "--force_skip_stats --force_skip_trans"
)

print(lp_update_cmd)
for cmd in crt_update_cmds:
    print(cmd)

python agencydata\wh_lp_update.py --log --force_unload --deal_type JUMBO2_0 --deal_list "CHASE 2024-6" --deal_downloaddate_start 20250901 --deal_downloaddate_end 20251001 --force_skip_stats --force_skip_trans
python agencydata\wh_crt_update.py --log --force_download --deal_type CAS --unload_all_collat --skip_update --skip_insert --force_save_stats
python agencydata\wh_crt_update.py --log --force_download --deal_type STACR --unload_all_collat --skip_update --skip_insert --force_save_stats
python agencydata\wh_crt_update.py --log --force_download --deal_type MIR --unload_all_collat --skip_update --skip_insert --force_save_stats
python agencydata\wh_crt_update.py --log --force_download --deal_type CAS_PSEUDO --unload_all_collat --skip_update --skip_insert --force_save_stats
python agencydata\wh_crt_update.py --log --force_download --deal_type STACR_PSEUDO --unload_all_collat --skip_update --skip_insert --force_save_stats


## 5. HELOC Data Updates
HELOC-specific flat file updates and operations

In [55]:
# HELOC Deal Lists
HELOC_DEALS = "GRADE 2021-FIG2,GRADE 2023-FIG3,GRADE 2023-FIG4,GRADE 2024-FIG5,FIGRE 2023-HE1,FIGRE 2023-HE1A2,FIGRE 2023-HE3,FIGRE 2024-HE1,FIGRE 2024-HE3,FIGRE 2024-HE6,FIGRE 2025-HE2,FIGRE 2025-HE3"

# HELOC Vector Generation
heloc_vector_cmd = f"python lmsimvectors\\lm_sim_pub_main.py -mode batch-ray -request_mode forward_proj -dbsavefromdll_json -dbsavefromdll_json_purpose HELOC_New -scenarios \"Base\" -bbg_deal_list \"{HELOC_DEALS}\" -as_of_date {AS_OF_DATE} -num_of_threads {NUM_THREADS} -num_of_workers {NUM_WORKERS} -force_rerun -ray_cluster {RAY_CLUSTER}"

print("HELOC Vector Generation:")
print(heloc_vector_cmd)
print()

# HELOC Data Updates
heloc_data_updates = [
    f"python lmdv01\\dv01_update.py --log --force_save_stats",
    f"python lmdv01\\dv01_update_platf.py --log --force_save_stats",
    f"python agencydata\\wh_lp_update.py --log --deal_type HELOC_PSEUDO --force_unload --deal_downloaddate_start 20190401 --deal_downloaddate_end {AS_OF_DATE[:-2]}01 --force_save_stats --force_save_trans"
]

print("HELOC Data Updates:")
for cmd in heloc_data_updates:
    print(cmd)
    print()

HELOC Vector Generation:
python lmsimvectors\lm_sim_pub_main.py -mode batch-ray -request_mode forward_proj -dbsavefromdll_json -dbsavefromdll_json_purpose HELOC_New -scenarios "Base" -bbg_deal_list "GRADE 2021-FIG2,GRADE 2023-FIG3,GRADE 2023-FIG4,GRADE 2024-FIG5,FIGRE 2023-HE1,FIGRE 2023-HE1A2,FIGRE 2023-HE3,FIGRE 2024-HE1,FIGRE 2024-HE3,FIGRE 2024-HE6,FIGRE 2025-HE2,FIGRE 2025-HE3" -as_of_date 20260128 -num_of_threads 7 -num_of_workers 16 -force_rerun -ray_cluster east-spot

HELOC Data Updates:
python lmdv01\dv01_update.py --log --force_save_stats

python lmdv01\dv01_update_platf.py --log --force_save_stats

python agencydata\wh_lp_update.py --log --deal_type HELOC_PSEUDO --force_unload --deal_downloaddate_start 20190401 --deal_downloaddate_end 20260101 --force_save_stats --force_save_trans



## 6. Monthly Data Refresh Commands
Comprehensive list of all data update commands to run monthly

In [56]:
# Single-deal quick updates
print("=== Single Deal Quick Updates ===")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"NONQM\" --deal_list \"VERUS 2022-1\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"HELOC\" --deal_list \"RCKT 2025-CES7\" --force_save_stats")
print("python lmdv01\\dv01_update.py --deal_list \"ACHM 2023-HE1\" --force_save_stats")
print("python agencydata\\wh_crt_update.py --deal_type CAS --deal_list \"CAS 2022-R02\" --force_save_stats")
print("python agencydata\\wh_crt_update.py --deal_type STACR --deal_list \"STACR 2019-DNA3\" --force_save_stats")
print()

print("=== Comprehensive Monthly Refresh ===")
print("# LP Updates by Deal Type")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"NONQM\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"HELOC\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"JUMBO2_0\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"SUBPRIME\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"ALT_A\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"SCRT\" --force_save_stats")
print()

print("# Pseudo Deal Updates")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"JUMBO2_0_PSEUDO\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"NONQM_PSEUDO\" --force_save_stats")
print("Python agencydata/wh_lp_update.py --skip_unload --deal_type \"HELOC_PSEUDO\" --force_save_stats")
print()

print("# Pseudo Deal Stats with Transitions")
print("python agencydata\\wh_lp_update.py --deal_type NONQM_PSEUDO --skip_unload --force_save_stats --stats_type pool_status --force_save_trans")
print("python agencydata\\wh_lp_update.py --deal_type JUMBO2_0_PSEUDO --skip_unload --force_save_stats --stats_type pool_status --force_save_trans")
print("python agencydata\\wh_lp_update.py --deal_type HELOC_PSEUDO --skip_unload --force_save_stats --stats_type pool_status --force_save_trans")
print()

print("# DV01 Updates")
print("python lmdv01\\dv01_update.py --force_save_stats")
print("python lmdv01\\dv01_update_platf.py --force_save_stats")
print()

print("# CRT Updates")
print("python agencydata\\wh_crt_update.py --deal_type MIR --force_save_stats")
print("python agencydata\\wh_crt_update.py --deal_type CAS --force_save_stats")
print("python agencydata\\wh_crt_update.py --deal_type STACR --force_save_stats")
print()

print("# CRT Pseudo Updates with Different Stats Types")
print("python agencydata\\wh_crt_update.py --deal_type CAS_PSEUDO --skip_update --skip_unload --force_save_stats --stats_type pool_status --force_save_trans")
print("python agencydata\\wh_crt_update.py --deal_type CAS_PSEUDO --skip_update --skip_unload --force_save_stats --stats_type adv_pool --force_save_trans")
print("python agencydata\\wh_crt_update.py --deal_type STACR_PSEUDO --skip_update --skip_unload --force_save_stats --stats_type pool_status --force_save_trans")
print("python agencydata\\wh_crt_update.py --deal_type STACR_PSEUDO --skip_update --skip_unload --force_save_stats --stats_type adv_pool --force_save_trans")

=== Single Deal Quick Updates ===
Python agencydata/wh_lp_update.py --skip_unload --deal_type "NONQM" --deal_list "VERUS 2022-1" --force_save_stats
Python agencydata/wh_lp_update.py --skip_unload --deal_type "HELOC" --deal_list "RCKT 2025-CES7" --force_save_stats
python lmdv01\dv01_update.py --deal_list "ACHM 2023-HE1" --force_save_stats
python agencydata\wh_crt_update.py --deal_type CAS --deal_list "CAS 2022-R02" --force_save_stats
python agencydata\wh_crt_update.py --deal_type STACR --deal_list "STACR 2019-DNA3" --force_save_stats

=== Comprehensive Monthly Refresh ===
# LP Updates by Deal Type
Python agencydata/wh_lp_update.py --skip_unload --deal_type "NONQM" --force_save_stats
Python agencydata/wh_lp_update.py --skip_unload --deal_type "HELOC" --force_save_stats
Python agencydata/wh_lp_update.py --skip_unload --deal_type "JUMBO2_0" --force_save_stats
Python agencydata/wh_lp_update.py --skip_unload --deal_type "SUBPRIME" --force_save_stats
Python agencydata/wh_lp_update.py --skip_u

---
# 📊 VECTOR GENERATION
Generate LMSim vectors, tracking reports, and position-only runs

---

## 7. Tracking Vectors & Reports
Core simulation operations including ECA, Matrix, and Forward Projections

In [6]:
# Standard LMSim Command Generator Functions
def generate_lmsim_commands(deal_type, as_of_date, purpose, scenarios="Base",
                            ray_cluster="east", disable_shock=False, deal_list=None):
    """Generate standard LMSim command sequence (Windows-safe quoting)."""
    base_cmd = f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray"

    disable_shock_str = "-disable_shock" if disable_shock else ""
    deal_list_str = f'-bbg_deal_list "{deal_list}"' if deal_list else ""

    commands = {
        "forward_proj": (
            f"{base_cmd} -request_mode forward_proj -save_transition -dbsavefromdll_json "
            f"-dbsavefromdll_json_purpose {purpose}_tracking -scenarios \"{scenarios}\" "
            f"-deal_type {deal_type} -force_rerun -ray_cluster {ray_cluster} "
            f"{disable_shock_str} {deal_list_str}"
        ),
        "eca": (
            f"{base_cmd} -request_mode eca -scenarios \"{scenarios}\" "
            f"-deal_type {deal_type} -dbsavefromdll_json_purpose {purpose} "
            f"-force_rerun -ray_cluster {ray_cluster} {disable_shock_str} {deal_list_str}"
        ),
        "matrix": (
            f"{base_cmd} -request_mode matrix -scenarios \"{scenarios}\" "
            f"-deal_type {deal_type} -dbsavefromdll_json_purpose {purpose} "
            f"-force_rerun -ray_cluster {ray_cluster} {disable_shock_str} {deal_list_str}"
        ),
    }
    return commands



def generate_tracking_command(deal_type, as_of_date, tracking_purpose,
                              email="hzeng@libremax.com", ignore_dials=False):
    return (
        f"python lmanalytics\\tracking\\transition_report.py --rebuild --date {as_of_date} "
        f"-m {email} --tracking_purpose {tracking_purpose} --purpose {tracking_purpose}_tracking "
        f"--deal_type {deal_type} {'--ignore_dials' if ignore_dials else ''} --dev"
    )


# Example: Generate commands for a deal type
deal_type = "STACR_PSEUDO,CAS_PSEUDO,HELOC_PSEUDO,JUMBO2_0_PSEUDO,NONQM_PSEUDO"  # STACR_PSEUDO,CAS_PSEUDO,HELOC_PSEUDO,JUMBO2_0_PSEUDO,NONQM_PSEUDO 
deal_list = None
AS_OF_DATE_EXAMPLE = "20260211"
purpose = "tracking_hz"
disable_shock = False
ray_cluster = "east"

commands = generate_lmsim_commands(
    deal_type, AS_OF_DATE_EXAMPLE, purpose,
    ray_cluster=ray_cluster,
    deal_list=deal_list,
    disable_shock=disable_shock,
)

commands["tracking"] = generate_tracking_command(
    deal_type, AS_OF_DATE_EXAMPLE, purpose, ignore_dials=disable_shock
)

for operation, cmd in commands.items():
    print(f"{operation.upper()} COMMAND:")
    print(cmd)
    print()

FORWARD_PROJ COMMAND:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode forward_proj -save_transition -dbsavefromdll_json -dbsavefromdll_json_purpose tracking_hz_tracking -scenarios "Base" -deal_type STACR_PSEUDO,CAS_PSEUDO,HELOC_PSEUDO,JUMBO2_0_PSEUDO,NONQM_PSEUDO -force_rerun -ray_cluster east  

ECA COMMAND:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode eca -scenarios "Base" -deal_type STACR_PSEUDO,CAS_PSEUDO,HELOC_PSEUDO,JUMBO2_0_PSEUDO,NONQM_PSEUDO -dbsavefromdll_json_purpose tracking_hz -force_rerun -ray_cluster east  

MATRIX COMMAND:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode matrix -scenarios "Base" -deal_type STACR_PSEUDO,CAS_PSEUDO,HELOC_PSEUDO,JUMBO2_0_PSEUDO,NONQM_PSEUDO -dbsavefromdll_json_purpose tracking_hz -force_rerun -ray_cluster east  

TRACKING COMMAND:
python lmanalytics\tracking\transition_report.py --rebuild --date 20260211 -m hzeng@

## 8. Ad-hoc LMSim Commands
Quick one-off LMSim commands for testing and specific analyses

In [2]:
# Dynamic as_of_date for all commands
as_of_date = "20260211"  # Set this once and all commands below use it

# Position-Only ECA Run
print("Position-Only ECA:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode eca -scenarios \"Base\" -position_only -position_asofdate {as_of_date} -deal_type JUMBO2_0 -dbsavefromdll_json_purpose PROD -force_rerun -ray_cluster east")
print()

# Jumbo with Specific Deal List and Scenarios
print("Jumbo with Deal List:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Jumbo_Inv -scenarios \"Base,ParallelDn200,ParallelUp200\" -deal_type JUMBO2_0 -bbg_deal_list \"CHASE 2024-6,CHASE 2024-9,JPMMT 2025-INV1,JPMMT 2024-INV1,RCKT 2024-INV1,GSMBS 2024-INV1,GCAT 2024-INV3,PMTLT 2025-INV6\" -ray_cluster east")
print()

# MI Position-Only
print("MI Position-Only:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_crt_mi -scenarios \"Base\" -deal_type MI -position_only -position_asofdate {as_of_date}")
print()

# Jumbo with Scenario Batch (default+opera)
print("Jumbo with default+opera scenarios:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_jumbo_v3 -scenarios_batch \"default+opera\" -deal_type JUMBO2_0 -position_only -position_asofdate {as_of_date}")
print()

# Multi Deal Type Position-Only
print("Multi Deal Type Position-Only:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Jumbo_Inv -scenarios_batch \"default+opera\" -deal_type JUMBO2_0 -position_only -position_asofdate {as_of_date} -ray_cluster east")
print()

# Full Universe with Scenario Batch
print("Full Universe with Scenario Batch:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose PROD -scenarios_batch \"default+opera\" -deal_type CRT,MI,JUMBO2_0,NONQM,HELOC -position_only -position_asofdate {as_of_date} -ray_cluster east-spot")
print()

# CRT,NONQM,HELOC,MI Position-Only
print("CRT,NONQM,JUMBO2_0,HELOC,MI Position-Only, Base:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose PROD_DIAL_V2 -scenarios_batch \"default+opera\" -deal_type CRT,MI,JUMBO2_0,NONQM,HELOC -position_only -position_asofdate {as_of_date}")
print()

# Jumbo PROD with Scenario Batch
print("Jumbo PROD with Scenario Batch:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose PROD -scenarios_batch \"default+opera\" -deal_type JUMBO2_0 -position_only -position_asofdate {as_of_date}")
print()

## 
print("Jumbo with Deal List:")
print(f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose PROD -scenarios_batch \"default+opera\" -deal_type HELOC -bbg_deal_list \"\" -ray_cluster east")
print()

print("Investor Jumbo Deal")
deal_list = [
    "PMTLT 2025-INV6",
    "GCAT 2024-INV3",
    "JPMMT 2025-INV1",
    "GSMBS 2024-INV1",
    "RCKT 2024-INV1",
    "JPMMT 2024-INV1",
]
deal_list_str = ",".join(deal_list)
print(
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} "
    f"-mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json "
    f"-dbsavefromdll_json_purpose PROD_INVESTOR -scenarios_batch \"default+opera\" "
    f"-deal_type JUMBO2_0 -bbg_deal_list \"{deal_list_str}\" -ray_cluster east"
)
print()
## CRT,NONQM,HELOC,

Position-Only ECA:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode eca -scenarios "Base" -position_only -position_asofdate 20260211 -deal_type JUMBO2_0 -dbsavefromdll_json_purpose PROD -force_rerun -ray_cluster east

Jumbo with Deal List:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Jumbo_Inv -scenarios "Base,ParallelDn200,ParallelUp200" -deal_type JUMBO2_0 -bbg_deal_list "CHASE 2024-6,CHASE 2024-9,JPMMT 2025-INV1,JPMMT 2024-INV1,RCKT 2024-INV1,GSMBS 2024-INV1,GCAT 2024-INV3,PMTLT 2025-INV6" -ray_cluster east

MI Position-Only:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260211 -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_crt_mi -scenarios "Base" -deal_type MI -position_only -position_asofdate 20260211

Jumbo with default+opera scenario

In [33]:
# Configurable Position-Only Command Generator
as_of_date = "20260114"
purpose = "PROD"
scenarios_batch = "default+opera"
deal_type = "JUMBO2_0"
ray_cluster = "east-spot"

cmd = (
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date} "
    f"-mode batch-ray -request_mode forward_proj -dbsavefromdll_json "
    f"-dbsavefromdll_json_purpose {purpose} -scenarios_batch {scenarios_batch} "
    f"-deal_type {deal_type} -position_only -position_asofdate {as_of_date} -ray_cluster {ray_cluster}"
)
print("Configurable Command:")
print(cmd)
print()

# CRT Specific Purpose
as_of_date_crt = "20251010"
purpose_crt = "Howard_crt_FCLStoP"
scenarios_batch_crt = "default+opera"
deal_type_crt = "CAS,STACR"

cmd_crt = (
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {as_of_date_crt} "
    f"-mode batch-ray -request_mode forward_proj -dbsavefromdll_json "
    f"-dbsavefromdll_json_purpose {purpose_crt} -scenarios_batch {scenarios_batch_crt} "
    f"-deal_type {deal_type_crt} -position_only -position_asofdate {as_of_date_crt} -ray_cluster {ray_cluster}"
)
print("CRT Specific Command:")
print(cmd_crt)

Configurable Command:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260114 -mode batch-ray -request_mode forward_proj -dbsavefromdll_json -dbsavefromdll_json_purpose PROD -scenarios_batch default+opera -deal_type JUMBO2_0 -position_only -position_asofdate 20260114 -ray_cluster east-spot

CRT Specific Command:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20251010 -mode batch-ray -request_mode forward_proj -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_crt_FCLStoP -scenarios_batch default+opera -deal_type CAS,STACR -position_only -position_asofdate 20251010 -ray_cluster east-spot


## 9. Position-Only and Weekend Runs
Position-only vector generation and weekend operations

In [34]:
# Position-Only Vector Generation
AS_OF_DATE_POS = 20260209
purpose = "Howard_jumbo_new_v1"
position_only_commands = [
    # Daily position-only run
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch \"default+opera\" -deal_type JUMBO2_0 -position_only -position_asofdate {AS_OF_DATE_POS}",

    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch \"default+opera\" -deal_type CRT,JUMBO2_0,NONQM,HELOC -position_only -position_asofdate {AS_OF_DATE_POS}",
    
    # Full universe weekend run
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch weekend -deal_type CRT,JUMBO2_0,NONQM,HELOC,MI -exclude_position -position_asofdate {AS_OF_DATE_POS}",

    # HELOC weekend run
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch weekend -deal_type CRT,JUMBO2_0,NONQM,HELOC,MI -exclude_position -position_asofdate {AS_OF_DATE_POS}",

    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch weekend -deal_type CRT,JUMBO2_0,NONQM,HELOC,MI -exclude_position -position_asofdate {AS_OF_DATE_POS}",
    
    # One Deal Type Only
    f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE_POS} -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios_batch weekend -deal_type MI -exclude_position -position_asofdate {AS_OF_DATE_POS}"
]

print("Position-Only Commands:")
for i, cmd in enumerate(position_only_commands, 1):
    print(f"Command {i}:")
    print(cmd)
    print()

Position-Only Commands:
Command 1:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260209 -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_jumbo_new_v1 -scenarios_batch "default+opera" -deal_type JUMBO2_0 -position_only -position_asofdate 20260209

Command 2:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260209 -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_jumbo_new_v1 -scenarios_batch "default+opera" -deal_type CRT,JUMBO2_0,NONQM,HELOC -position_only -position_asofdate 20260209

Command 3:
python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260209 -mode batch-ray -request_mode forward_proj -dbsavefromdll_legacy -dbsavefromdll_json -dbsavefromdll_json_purpose Howard_jumbo_new_v1 -scenarios_batch weekend -deal_type CRT,JUMBO2_0,NONQM,HELOC,MI -exclude_position -position_asofdate 20260209

Command 4:
python lmsimvectors\lm_sim_pub_m

## 10. SSS and Tape File Run

In [24]:
# --- Tape & SSS File Processing ---
# Example TAP/SSS vector generation and CRT deal manager invocation

AS_OF_DATE_TAP_SSS = 20260203
DEAL_LIST = "GRADE 2026-HB1"
PROD_PURPOSE = "PROD"
SCENARIOS = "Base,Bear,Bull,SuperBear,SuperBull"

# Run vector generation for the SSS tape file
cmd_tape_file = (
    f"python lmsimvectors\\lm_sim_pub_main.py "
    f"-as_of_date {AS_OF_DATE_TAP_SSS} -mode batch-ray -request_mode forward_proj "
    f"-dbsavefromdll_json -dbsavefromdll_json_purpose {PROD_PURPOSE} "
    f"-scenarios \"{SCENARIOS}\" "
    f"-bbg_deal_list \"{DEAL_LIST}\""
)
print("TAP/SSS Tape File Command:\n", cmd_tape_file)
print("\t")

# Define scenario sets
TEST_SCENS = ['Base', 'Bear', 'Bull', 'SuperBear', 'SuperBull']  # For crt_deal_manager_config.py

# Run CRT deal manager for this deal and scenario set
crt_deal_manager_cmd = (
    f"python lmsimvectors\\crt_deal_manager_main.py "
    f"-asofdate {AS_OF_DATE_TAP_SSS} -bbg_deal_list \"{DEAL_LIST}\" "
    f"-purpose {PROD_PURPOSE} -scenarioset TEST_SCENS"
)
print("CRT Deal Manager Command:\n", crt_deal_manager_cmd)

print("\t")
print("Test scenario sets for CRT manager config (full):", TEST_SCENS)


TAP/SSS Tape File Command:
 python lmsimvectors\lm_sim_pub_main.py -as_of_date 20260203 -mode batch-ray -request_mode forward_proj -dbsavefromdll_json -dbsavefromdll_json_purpose PROD -scenarios "Base,Bear,Bull,SuperBear,SuperBull" -bbg_deal_list "GRADE 2026-HB1"
	
CRT Deal Manager Command:
 python lmsimvectors\crt_deal_manager_main.py -asofdate 20260203 -bbg_deal_list "GRADE 2026-HB1" -purpose PROD -scenarioset TEST_SCENS
	
Test scenario sets for CRT manager config (full): ['Base', 'Bear', 'Bull', 'SuperBear', 'SuperBull']


---
# 📈 RISK OPERATIONS
Generate risk vectors and portfolio risk analysis

---

## 10. Risk Run Operations
Generate risk vectors and JSON outputs

Be sure to change lmmodel\config\crtmodel_config.py json_purpose

In [5]:
# Risk Run Commands
def generate_risk_run(date, scenario_set, cusips, purpose, num_processes=8):
    return f"python RiskRun\\riskrun_main.py -d {date} --scenarioset {scenario_set} -c {cusips} --save --purpose {purpose} --numprocesses {num_processes}"

# Common CUSIP lists
AS_OF_DATE_RISK = 20260211
CRT_CUSIPS = "20753VDL9,20754KAJ0,20754BAJ0,20753UAF7,20753UAJ9,35564TBE9,35563MBE5,35565HBE4,35565LBE5,35564KLV9,35564KPP8,35564KZY8"
HELOC_CUSIPS = "FIGUREMEZZ,302455AD2,31684EAD9,31684CAE1,31684HAG5,31684HAF7,31684GAG7,31684GAF9,31684DAG4,31684DAH2,31684JAG1,31684JAH9,31684KAG8,31684KAH6,31684LAF8,FIGUREWL1,FIGUREWL2,79582AAD9,79582MAF8"
MIR_CUSIPS = "07876GAE0,437307AE1,43730GAD3,75049GAC3,75049XAB8,75049AAB8,89600HAG4,89600HAC3,89589AAB5,078774AD8,26982EAG1,43731BAD3,75050KAD8,89589AAD1,89589AAC3"
CRT_CUSIPS_FULL = "30711XBQ6,20753VDL9,20754RAJ5,20754KAJ0,20754BAJ0,20753YAL6,20754NAR6,20754MAL1,20753UAJ9,20754GAJ9,35563PKX6,35563PPP8,35563PRM3,35563PUB3,35563P6W4,35563PYB9,35563XBE1,35564TBE9,35564PAC2,35565EAE2,35565GAE7,35563MBE5,35564MBE4,35565CBE5,35565HBE4,35565LBE5,35564KBE8,35564KLV9,35564KNE5,35564KCP2,35564KKL2,35564KPP8,35564KSQ3,35564KWG0,35564KZY8,35564KUL1"
JUMBO_CUSIPS = "75023DCD6,362952BQ5,16160QAY0,46593RAJ5,89183GAE6,46593VAK3,46594CAK4,46594CAL2,46594CAM0,46594CAN8,89178BAJ3,161931AQ3,161600QAY0,161600CCC7,161935BB6,75023DBQ8,81750CDB8,16160BAC1,16159VBH8,161922BL2,3137FCJH8,36830FBA3,36295ZBQ5,36268VAF9,46591FAX2,46658TBH6,46657WAX6,46657WAY4,46658PAX0,466910AW6,61776QBQ8,69382MBK9,74938QBB9,75023DCDD6"
JUMBO_CUSIPS_SHORT = "36830FBA3,362952BQ5,46658PAX0,466910AW6,69382MBK9,74938QBB9"
Adholc_CUSIPS = "79582CAE3,79582CAF0"

# Generate risk run commands
risk_commands = {
    'CRT_Risk_Run': generate_risk_run(AS_OF_DATE_RISK, "RATE_HEDGE", CRT_CUSIPS, "Howard_crt_new"),
    'HELOC_Risk_Run': generate_risk_run(AS_OF_DATE_RISK, "RATE_HEDGE", HELOC_CUSIPS, "HELOC_New"),
    'MIR_Risk_Run': generate_risk_run(AS_OF_DATE_RISK, "RATE_HEDGE", MIR_CUSIPS, "Howard_crt_mir"),
    'CRT_Sectors': f"python RiskRun\\riskrun_main.py -d {AS_OF_DATE_RISK} --scenarioset RATE_HEDGE  --sectors CRT --numprocesses 8 --save --purpose Jumbo_Inv",
    "Adholc_Run": generate_risk_run(AS_OF_DATE_RISK, 'RATE_HEDGE', Adholc_CUSIPS, 'PROD_GRADE'),
    "Portfolio_Run": f"python RiskRun\\riskrun_main.py -d {AS_OF_DATE_RISK} --scenarioset PROD_DIAL_V2 --sectors CRT --numprocesses 8 --save --purpose QRPRODWRAPPERWITHSETTLE_CRT", # QRPRODWRAPPERWITHSETTLE_CRT
}

print("Risk Run Commands:")
for name, cmd in risk_commands.items():
    print(f"{name}:")
    print(cmd)
    print()

Risk Run Commands:
CRT_Risk_Run:
python RiskRun\riskrun_main.py -d 20260211 --scenarioset RATE_HEDGE -c 20753VDL9,20754KAJ0,20754BAJ0,20753UAF7,20753UAJ9,35564TBE9,35563MBE5,35565HBE4,35565LBE5,35564KLV9,35564KPP8,35564KZY8 --save --purpose Howard_crt_new --numprocesses 8

HELOC_Risk_Run:
python RiskRun\riskrun_main.py -d 20260211 --scenarioset RATE_HEDGE -c FIGUREMEZZ,302455AD2,31684EAD9,31684CAE1,31684HAG5,31684HAF7,31684GAG7,31684GAF9,31684DAG4,31684DAH2,31684JAG1,31684JAH9,31684KAG8,31684KAH6,31684LAF8,FIGUREWL1,FIGUREWL2,79582AAD9,79582MAF8 --save --purpose HELOC_New --numprocesses 8

MIR_Risk_Run:
python RiskRun\riskrun_main.py -d 20260211 --scenarioset RATE_HEDGE -c 07876GAE0,437307AE1,43730GAD3,75049GAC3,75049XAB8,75049AAB8,89600HAG4,89600HAC3,89589AAB5,078774AD8,26982EAG1,43731BAD3,75050KAD8,89589AAD1,89589AAC3 --save --purpose Howard_crt_mir --numprocesses 8

CRT_Sectors:
python RiskRun\riskrun_main.py -d 20260211 --scenarioset RATE_HEDGE  --sectors CRT --numprocesses 8 --sav

---
# 🔧 OTHER OPERATIONS
Utilities, debugging, database operations, and reference data

---

## 11. Development and Debug Operations
LMSim debug and development commands

In [62]:
# LMSim Debug Commands
lmsim_debug_prod = f"{GIT_LMSIMDATA_PROD_PATH}\\bin64\\LMIntexAppmdx64.exe /config-file={GIT_LMSIMDATA_PROD_PATH}\\input\\LMSim.json /define init_config=init_config\\init_norate.json /define run_scen={GIT_LMSIMDATA_PROD_PATH}\\input\\debug_input_files\\HowardTest.json /define common_config.data_root_folder={GIT_LMSIMDATA_PROD_PATH}\\ /define logging.channels.file_short1.path={GIT_LMSIMDATA_PROD_PATH}\\LMSim_debug.log /define logging.channels.file1.path={GIT_LMSIMDATA_PROD_PATH}\\LMSim_debug.log /define logging.loggers.root.channel=splitter1 /define debug.module_list=LMAsset.ResiAsset"

lmsim_debug_dev = f"{GIT_LMSIM_PATH}\\bin64\\LMIntexAppmdx64.exe /config-file={GIT_LMSIMDATA_PROD_PATH}\\input\\LMSim.json /define init_config=init_config\\init_norate.json /define run_scen={GIT_LMSIMDATA_PROD_PATH}\\input\\debug_input_files\\HowardTest.json /define common_config.data_root_folder={GIT_LMSIMDATA_PROD_PATH}\\ /define logging.channels.file_short1.path={GIT_LMSIM_PATH}\\LMSim_debug.log /define logging.channels.file1.path={GIT_LMSIM_PATH}\\LMSim_debug.log /define logging.loggers.root.channel=splitter1 /define debug.module_list=LMAsset.ResiAsset"

print("LMSim Debug Commands:")
print("Production Debug:")
print(lmsim_debug_prod)
print()
print("Development Debug:")
print(lmsim_debug_dev)
print()

LMSim Debug Commands:
Production Debug:
C:\Git\LMSimData_prod\bin64\LMIntexAppmdx64.exe /config-file=C:\Git\LMSimData_prod\input\LMSim.json /define init_config=init_config\init_norate.json /define run_scen=C:\Git\LMSimData_prod\input\debug_input_files\HowardTest.json /define common_config.data_root_folder=C:\Git\LMSimData_prod\ /define logging.channels.file_short1.path=C:\Git\LMSimData_prod\LMSim_debug.log /define logging.channels.file1.path=C:\Git\LMSimData_prod\LMSim_debug.log /define logging.loggers.root.channel=splitter1 /define debug.module_list=LMAsset.ResiAsset

Development Debug:
C:\Git\LMSim\bin64\LMIntexAppmdx64.exe /config-file=C:\Git\LMSimData_prod\input\LMSim.json /define init_config=init_config\init_norate.json /define run_scen=C:\Git\LMSimData_prod\input\debug_input_files\HowardTest.json /define common_config.data_root_folder=C:\Git\LMSimData_prod\ /define logging.channels.file_short1.path=C:\Git\LMSim\LMSim_debug.log /define logging.channels.file1.path=C:\Git\LMSim\LMSi

## 12. Utility Commands
Git operations, Ray job management, and other utilities

In [72]:
# Git Operations
GITHUB_TOKEN = "" # I have that stored in notebook
git_commands = [
    f"git clone -b crt_mir --single-branch https://{GITHUB_TOKEN}@github.com/LibreMax-QR/LMSim.git",
    f"git clone https://{GITHUB_TOKEN}@github.com/LibreMax-QR/LMSim.git"
]

print("Git Commands:")
for cmd in git_commands:
    print(cmd)
    print()

# Ray Job Management
job_id = "raysubmit_hffFzXXAgfFkT8A2"
ray_stop_commands = [
    f'curl -X POST "https://lmsim-ray.qr.libremax.com/api/jobs/{job_id}/stop" -k',
    f'curl -X POST "http://lmsim-ray-spot.qr.libremax.com/api/jobs/{job_id}/stop" -k',
    f'curl -X POST "https://lmsim-ray.qr-west3.libremax.com/api/jobs/{job_id}/stop" -k'
]

print("Ray Job Stop Commands:")
for cmd in ray_stop_commands:
    print(cmd)
    print()

Git Commands:
git clone -b crt_mir --single-branch https://@github.com/LibreMax-QR/LMSim.git

git clone https://@github.com/LibreMax-QR/LMSim.git

Ray Job Stop Commands:
curl -X POST "https://lmsim-ray.qr.libremax.com/api/jobs/raysubmit_hffFzXXAgfFkT8A2/stop" -k

curl -X POST "http://lmsim-ray-spot.qr.libremax.com/api/jobs/raysubmit_hffFzXXAgfFkT8A2/stop" -k

curl -X POST "https://lmsim-ray.qr-west3.libremax.com/api/jobs/raysubmit_hffFzXXAgfFkT8A2/stop" -k



## 13. Deal Lists and Constants
Commonly used deal lists and identifiers

In [1]:
# Common Deal Lists
DEAL_LISTS = {
    'STACR_RECENT': "STACR 2019-DNA3,STACR 2019-HQA1,STACR 2020-DNA1,STACR 2020-HQA2,STACR 2021-DNA6,STACR 2021-HQA4,STACR 2022-DNA5,STACR 2022-HQA1,STACR 2023-DNA1,STACR 2023-HQA1,STACR 2024-DNA3,STACR 2024-HQA2",
    
    'CAS_RECENT': "CAS 2019-R06,CAS 2020-R02,CAS 2021-R02,CAS 2022-R02,CAS 2023-R05,CAS 2024-R01",
    
    'HELOC_GRADE': "GRADE 2021-FIG2,GRADE 2023-FIG3,GRADE 2023-FIG4,GRADE 2024-FIG5",
    
    'HELOC_FIGRE': "FIGRE 2023-HE1,FIGRE 2023-HE1A2,FIGRE 2023-HE3,FIGRE 2024-HE1,FIGRE 2024-HE3,FIGRE 2024-HE6,FIGRE 2025-HE2,FIGRE 2025-HE3",
    
    'JUMBO_RECENT': "FSMT 2021-6INV,GCAT 2022-INV3,JPMMT 2021-INV7,JPMMT 2021-LTV2,JPMMT 2024-1,MELLO 2021-MTG1,OBX 2022-J1,SEMT 2023-1,SEMT 2025-6,TPMT 2024-4,UWM 2021-INV1",
    
    'NONQM_RECENT': "BRAVO 2021-NQM1,CHNGE 2022-1,COLT 2024-5,SAN 2025-NQM1,VISIO 2023-1,ADMT 2023-NQM3,CHNGE 2022-2,COLT 2022-3,PRKCM 2021-AFC1,SGR 2022-1,VERUS 2022-1,VERUS 2022-3"
}

print("Deal Lists:")
for category, deals in DEAL_LISTS.items():
    print(f"{category}:")
    print(deals)
    print()

Deal Lists:
STACR_RECENT:
STACR 2019-DNA3,STACR 2019-HQA1,STACR 2020-DNA1,STACR 2020-HQA2,STACR 2021-DNA6,STACR 2021-HQA4,STACR 2022-DNA5,STACR 2022-HQA1,STACR 2023-DNA1,STACR 2023-HQA1,STACR 2024-DNA3,STACR 2024-HQA2

CAS_RECENT:
CAS 2019-R06,CAS 2020-R02,CAS 2021-R02,CAS 2022-R02,CAS 2023-R05,CAS 2024-R01

HELOC_GRADE:
GRADE 2021-FIG2,GRADE 2023-FIG3,GRADE 2023-FIG4,GRADE 2024-FIG5

HELOC_FIGRE:
FIGRE 2023-HE1,FIGRE 2023-HE1A2,FIGRE 2023-HE3,FIGRE 2024-HE1,FIGRE 2024-HE3,FIGRE 2024-HE6,FIGRE 2025-HE2,FIGRE 2025-HE3

JUMBO_RECENT:
FSMT 2021-6INV,GCAT 2022-INV3,JPMMT 2021-INV7,JPMMT 2021-LTV2,JPMMT 2024-1,MELLO 2021-MTG1,OBX 2022-J1,SEMT 2023-1,SEMT 2025-6,TPMT 2024-4,UWM 2021-INV1

NONQM_RECENT:
BRAVO 2021-NQM1,CHNGE 2022-1,COLT 2024-5,SAN 2025-NQM1,VISIO 2023-1,ADMT 2023-NQM3,CHNGE 2022-2,COLT 2022-3,PRKCM 2021-AFC1,SGR 2022-1,VERUS 2022-1,VERUS 2022-3



## 14. Quick Command Generator
Generate commands dynamically based on current configuration

In [21]:
# Quick Command Generator Function
def generate_full_workflow(deal_type, purpose_suffix="test"):
    """Generate a complete workflow for a deal type"""
    purpose = f"Howard_{purpose_suffix}"
    
    workflow = {
        'data_update': generate_wh_crt_update(deal_type, date_ranges['unload_start'], date_ranges['unload_end']),
        'eca': f"python lmsimvectors\\lm_sim_pub_main.py -mode batch-ray -request_mode eca -scenarios \"{BASE_SCENARIOS}\" -deal_type {deal_type} -as_of_date {AS_OF_DATE} -dbsavefromdll_json_purpose {purpose} -force_rerun -ray_cluster {RAY_CLUSTER}",
        'matrix': f"python lmsimvectors\\lm_sim_pub_main.py -mode batch-ray -request_mode matrix -scenarios \"{BASE_SCENARIOS}\" -deal_type {deal_type} -as_of_date {AS_OF_DATE} -dbsavefromdll_json_purpose {purpose} -force_rerun -ray_cluster {RAY_CLUSTER}",
        'forward_proj': f"python lmsimvectors\\lm_sim_pub_main.py -as_of_date {AS_OF_DATE} -mode batch-ray -request_mode forward_proj -save_transition -dbsavefromdll_json -dbsavefromdll_json_purpose {purpose} -scenarios \"{TRACKING_SCENARIOS}\" -deal_type {deal_type} -force_rerun -ray_cluster {RAY_CLUSTER}",
        'tracking': f"python lmanalytics\\tracking\\transition_report.py --rebuild --date {AS_OF_DATE} -m {EMAIL_RECIPIENTS} --tracking_purpose {purpose} --purpose {purpose} --deal_type {deal_type}"
    }
    
    return workflow

# Example: Generate workflow for CAS_PSEUDO
cas_workflow = generate_full_workflow("CAS_PSEUDO", "cas_test")

print(f"Complete CAS_PSEUDO Workflow for {AS_OF_DATE}:")
for step, command in cas_workflow.items():
    print(f"{step.upper()}:")
    print(command)
    print()

NameError: name 'generate_wh_crt_update' is not defined

## 15. Google Cloud Access

Go here: https://console.cloud.google.com/welcome?inv=1&invt=Ab27lg&project=data-direct-34fx

Use credentials in "S:\QR\passwd\dv01_bigquery.txt"

"SQL Email"

Then say "run a big query"

Under "libremax", select the table you want to query...choose 3 dots and "sql query"

## 16. Tape Cracking
Tape cracker commands for new deal onboarding

In [20]:
AS_OF_DATE = 20260202
cmd = (
    f'python agencydata\\run_tape_cracker.py --deal_name "GRADE 2026-HB1" '
    f'--deal_type "FIGURE" --pool_id GRADE2026HB1 --asofdate {AS_OF_DATE} '
    f'--src_data_dir "S:\\QR\\TapeCracker\\template_V1" '
    f'--file_name "tape_cracker_V1.5_GRADE 2026-HB1 - Copy.xlsx" '
    f'--output_data_dir "\\\\lmaxquantstorage.file.core.windows.net\\lmaxqr\\FlatFilesMonthly\\HELOCMonthly" '
    f'--save_src_to_db --save_stats_to_db'
)
print(cmd)

python agencydata\run_tape_cracker.py --deal_name "GRADE 2026-HB1" --deal_type "FIGURE" --pool_id GRADE2026HB1 --asofdate 20260202 --src_data_dir "S:\QR\TapeCracker\template_V1" --file_name "tape_cracker_V1.5_GRADE 2026-HB1 - Copy.xlsx" --output_data_dir "\\lmaxquantstorage.file.core.windows.net\lmaxqr\FlatFilesMonthly\HELOCMonthly" --save_src_to_db --save_stats_to_db


## 17. Database Operations

### Kill Running Queries

In [67]:
# SQL queries to find and kill running processes
sql_kill_queries = '''
-- Find running queries matching 'mir_core_dev'
SELECT * FROM stv_recents WHERE query LIKE '%mir_core_dev%';

SELECT * FROM stv_recents WHERE status = 'Running';

-- Cancel the query by process ID (PID)
CANCEL 1073890024;
'''

print(sql_kill_queries)


-- Find running queries matching 'mir_core_dev'
SELECT * FROM stv_recents WHERE query LIKE '%mir_core_dev%';

SELECT * FROM stv_recents WHERE status = 'Running';

-- Cancel the query by process ID (PID)
CANCEL 1073890024;



### Pool Mapping Queries

In [68]:
# Pool mapping queries
sql_pool_mapping = '''
-- Check pool group map
select * from lib.poolgroupmap where bb_deal like '%CHASE 2025-12%'

-- Check LP deal group
select * from lps.lp_deal_group_all where bb_deal = 'CHASE 2025-12' limit 10;
'''

print(sql_pool_mapping)


-- Check pool group map
select * from lib.poolgroupmap where bb_deal like '%CHASE 2025-12%'

-- Check LP deal group
select * from lps.lp_deal_group_all where bb_deal = 'CHASE 2025-12' limit 10;



## 18. IntexLoader Operations
Commands for loading Intex data into the database

In [69]:
# Simple IntexLoader command
print("Simple IntexLoader Example:")
print("python lmdata\\lmloader\\rmbsintexloader.py --startdate 2025101 --enddate 20260101 --save --deallist \"VRS2201\"")
print()

Simple IntexLoader Example:
python lmdata\lmloader\rmbsintexloader.py --startdate 2025101 --enddate 20260101 --save --deallist "VRS2201"



In [70]:
# IntexLoader with database query integration
from lmdata.lmdb import executeBulkSQLWithDF, executeSQL, executeSQLTableDF, get_engine, get_metadata
import lmdata.lmdb as lmdb

sql_string = """
    select distinct IntexDealName from libremax.dbo.IntexDealTS intex
    join libremax.dbo.CRTStats st on intex.BloombergDealName = st.bbg_deal_name
"""

df_intex_deal = lmdb.executeSQLTableDF(db="libremax", sql_query=sql_string, redshift=False)

In [71]:
def generate_rmbs_intexloader_cmd(startdate, enddate, deallist, save=True):
    cmd = f"python lmdata\\lmloader\\rmbsintexloader.py --startdate {startdate} --enddate {enddate} --deallist \"{deallist}\""
    if save:
        cmd += " --save"
    return cmd

def chunk_list(items, n_chunks=3):
    items = list(items)
    n = len(items)
    chunk_size = (n + n_chunks - 1) // n_chunks  # ceil
    return [items[i:i+chunk_size] for i in range(0, n, chunk_size)]

# Example usage:
startdate = "20251201"
enddate = "20260201"

deals = df_intex_deal['IntexDealName'].dropna().astype(str).tolist()
chunks = chunk_list(deals, n_chunks=3)

cmds = [
    generate_rmbs_intexloader_cmd(startdate, enddate, ",".join(chunk))
    for chunk in chunks
]

for i, cmd in enumerate(cmds, 1):
    print(f"CMD {i}:\n{cmd}\n")

CMD 1:
python lmdata\lmloader\rmbsintexloader.py --startdate 20251201 --enddate 20260201 --deallist "AGBY1301,AOAK1906,AOAK2002,ARMT0602,BSAA0510,BARC23N3,BRFT24N8,CML21IN1,COT24IN2,CSM06004,CSM21IN1,CWA05059,GCAT22N2,SGTM23S3,GSAA0604,GSMB21I2,GSMB22P5,GSMB23P3,GSMB24P4,IMF21NQ2,JPM20004,JPM21009,JPM22IN1,JPM24CC1,LXS0602N,MMC21IN1,MSRM2302,NWR21N2R,OBX21IN3,OBX22NQ1,PFMT2001,PRPM25N1,PSMC212,RATE21J3,RCKT24C2,SQMT1704,SQMT2108,SQMT2402,SQMT2503,SHM22SH1,SQMT25S1,ST19DNA3,ST21DNA3,ST21DNA7,ST22DNA3,UWM21I2,VRS21R1,VRS2203,VISIO221,VRS2506,VISIO192,WINW1403,AOAK1905,AOAK2101,BARC24N1,BRFT1901,BRFT23N1,BRFT23N8,CTAS17C7,CTAS21R3,CIM18I1,CIM19J1,CIM21I1,CML21J01,COT21HX1,COT2304,CROS23H2,CSM19NQ1,CITDL201,EFMT202,EFMT221,FIGRE243,FLGM182,FLGM186,FLGM2112,GCAT22N3,GSMB19P1,GSMB21P7,GSMB21P8,HOME24N1,IMF20NQ1,JPM16005,JPM20009,JPM21011,MSRM2501,OBX20EX3,OBX24N13,OBX25NQ6,PFMT20F1,PFMT2502,RMLT211R,ST18HRP1,VRS2108,VRS25I1,AGBY1505,AOAK2104,BARC23N2,BRFT21N3,CTAS25R6,COT2105,CSM15002,CSM19A